# SpaceX Interactive Dashboard with Plotly Dash

In this lab, we are building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component\
TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown\
TASK 3: Add a Range Slider to Select Payload\
TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

This is a part of the [IBM Data Science Professional Certificate](https://www.coursera.org/professional-certificates/ibm-data-science)

In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import wget

In [ ]:
# Retrieve ead the launch data into a pandas dataframe
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

In [ ]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div(dcc.Dropdown(id='site-dropdown', 
                                    options=[ 
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}], 
                                    value='ALL',
                                    placeholder="Select a launch site here",
                                    searchable=True
                                )),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                min = 0,
                                max = 10000,
                                step = 1000,
                                marks = {0:'0',2500:'2,500 kg',5000:'5,000 kg',7500:'7,500 kg',10000: '10,000 kg'},
                                value = [min_payload,max_payload])),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        # return the outcomes piechart for a selected siteLaunch Site
        fig = px.pie(filtered_df, 
        names='class', 
        title='Total Success Launches for Site ' + entered_site)
        return fig

In [ ]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider', component_property='value')])

def get_scatter_plot(entered_site, payloads):
    if entered_site == 'ALL':
        filter_low_df = spacex_df[spacex_df['Payload Mass (kg)'] >= payloads[0]]
        filtered_df = filter_low_df[filter_low_df['Payload Mass (kg)'] <= payloads[1]]
        fig2 = px.scatter(filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        symbol='Booster Version Category',
        title='Correlation between payload and success for all sites between ' + str(payloads[0]) + 'kg and ' + str(payloads[1]) + 'kg')
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filter_low_df = site_df[site_df['Payload Mass (kg)'] >= payloads[0]]
        filtered_df = filter_low_df[filter_low_df['Payload Mass (kg)'] <= payloads[1]]
        fig2 = px.scatter(filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        symbol='Booster Version Category',
        title='Correlation between payload and success for ' + entered_site + ' between ' + str(payloads[0]) + 'kg and ' + str(payloads[1]) + 'kg')
    return fig2

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

To view the dashboard open a browser window at http://127.0.0.1:8050/